In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Laddar in datasetet.
df = pd.read_csv("winequality-red.csv", sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
# --- PLATSHÅLLARE (Ta bort och använd din egen inläsning) ---
from sklearn.datasets import load_wine
data_raw = load_wine()
df = pd.DataFrame(data_raw.data, columns=data_raw.feature_names)
df['target'] = data_raw.target
# ------------------------------------------------------------

In [4]:
# Dela upp i Features (X) och Target (y)
target_col = 'target' 
X_original = df.drop(target_col, axis=1)
y = df[target_col]

In [5]:
# Skapa normaliserad data (Viktigt för SVM!)
scaler = MinMaxScaler()
X_norm = pd.DataFrame(scaler.fit_transform(X_original), columns=X_original.columns)

In [6]:
# =============================================================================
# 2. KONFIGURATIONER (Spelreglerna)
# =============================================================================
# Vi ska testa kombinationer av:
# a) Data: Original vs Normaliserad
# b) Split: 75/25 vs Eget val (här 60/40)
# c) Kernel: 'linear' vs 'rbf' (Hur den drar gränserna)

datasets = [("Original", X_original), ("Normaliserad", X_norm)]
splits = [0.25, 0.40] # 25% test (75% träning) och 40% test (60% träning)
kernels = ['linear', 'rbf'] # De två sätten att rita gränser

alla_resultat = []

print("Startar SVM-körningar...\n")

Startar SVM-körningar...



In [7]:
# =============================================================================
# 3. KÖR LOOPARNA
# =============================================================================

# Loop 1: Data (Original/Normaliserad)
for data_name, X_data in datasets:
    
    # Loop 2: Split (Hur mycket plugg vs prov?)
    for test_size in splits:
        
        # Loop 3: Kernel (Vilken matematik ska användas?)
        for kernel_type in kernels:
            
            # 1. Dela upp data (Vi gör det en gång per kombination här)
            # Vill du ha det superstabilt kan du lägga en loop här (som i u2)
            # men uppgiften kräver inte uttryckligen 100 körningar här.
            X_train, X_test, y_train, y_test = train_test_split(
                X_data, y, test_size=test_size, random_state=42
            )
            
            # 2. Skapa modellen
            # Vi stoppar in vald kernel ('linear' eller 'rbf')
            svm_model = SVC(kernel=kernel_type, random_state=42)
            
            # 3. Träna
            svm_model.fit(X_train, y_train)
            
            # 4. Testa
            y_pred = svm_model.predict(X_test)
            
            # 5. Utvärdera
            acc = accuracy_score(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
            
            # Spara resultatet
            beskrivning = f"Data: {data_name}, Split: {int((1-test_size)*100)}/{int(test_size*100)}, Kernel: {kernel_type}"
            
            alla_resultat.append({
                "beskrivning": beskrivning,
                "accuracy": acc,
                "cm": cm
            })

In [8]:
# =============================================================================
# 4. PRESENTERA RESULTAT
# =============================================================================
print("-" * 60)
print("ALLA RESULTAT (Lista)")
print("-" * 60)

for res in alla_resultat:
    print(res['beskrivning'])
    print(f"Accuracy: {res['accuracy']:.4f}")
    print("Confusion Matrix:")
    print(res['cm'])
    print("-" * 20)

------------------------------------------------------------
ALLA RESULTAT (Lista)
------------------------------------------------------------
Data: Original, Split: 75/25, Kernel: linear
Accuracy: 0.9778
Confusion Matrix:
[[15  0  0]
 [ 0 17  1]
 [ 0  0 12]]
--------------------
Data: Original, Split: 75/25, Kernel: rbf
Accuracy: 0.7111
Confusion Matrix:
[[15  0  0]
 [ 0 13  5]
 [ 0  8  4]]
--------------------
Data: Original, Split: 60/40, Kernel: linear
Accuracy: 0.9444
Confusion Matrix:
[[26  0  0]
 [ 4 23  0]
 [ 0  0 19]]
--------------------
Data: Original, Split: 60/40, Kernel: rbf
Accuracy: 0.6528
Confusion Matrix:
[[23  0  3]
 [ 1 17  9]
 [ 1 11  7]]
--------------------
Data: Normaliserad, Split: 75/25, Kernel: linear
Accuracy: 1.0000
Confusion Matrix:
[[15  0  0]
 [ 0 18  0]
 [ 0  0 12]]
--------------------
Data: Normaliserad, Split: 75/25, Kernel: rbf
Accuracy: 1.0000
Confusion Matrix:
[[15  0  0]
 [ 0 18  0]
 [ 0  0 12]]
--------------------
Data: Normaliserad, Split: 60

In [9]:
# =============================================================================
# 5. TOPP 3 BÄSTA
# =============================================================================
# Sortera listan så högst accuracy kommer först
sorterade = sorted(alla_resultat, key=lambda x: x['accuracy'], reverse=True)

print("*" * 60)
print("DE 3 BÄSTA SVM-MODELLERNA")
print("*" * 60)

for i in range(min(3, len(sorterade))):
    res = sorterade[i]
    print(f"PLATS {i+1}: {res['beskrivning']}")
    print(f"Accuracy: {res['accuracy']:.4f}")
    print("Confusion Matrix:")
    print(res['cm'])
    print("-" * 30)

************************************************************
DE 3 BÄSTA SVM-MODELLERNA
************************************************************
PLATS 1: Data: Normaliserad, Split: 75/25, Kernel: linear
Accuracy: 1.0000
Confusion Matrix:
[[15  0  0]
 [ 0 18  0]
 [ 0  0 12]]
------------------------------
PLATS 2: Data: Normaliserad, Split: 75/25, Kernel: rbf
Accuracy: 1.0000
Confusion Matrix:
[[15  0  0]
 [ 0 18  0]
 [ 0  0 12]]
------------------------------
PLATS 3: Data: Normaliserad, Split: 60/40, Kernel: linear
Accuracy: 0.9861
Confusion Matrix:
[[26  0  0]
 [ 1 26  0]
 [ 0  0 19]]
------------------------------


In [ ]:
#1. Vad är SVM och Kernel?Tänk dig att du har röda och blå kulor på ett golv. Din uppgift är att lägga ett snöre mellan dem så de skiljs åt.Linear kernel: Du måste lägga snöret spikrakt (som en linjal). 
#Det funkar om kulorna ligger prydligt på varsin sida.RBF kernel: Du får lägga snöret i cirklar eller kurvor. 
#Det är bra om de röda kulorna ligger i en klunga mitt i de blåa.Uppgiften: Vi ska testa båda sätten ("Linear" och "RBF") för att se vad som passar din data bäst.
#2. Normalisering är superviktigt här!I förra uppgiften (Decision Trees) spelade det ingen roll om du mätte i millimeter eller kilometer. 
# Men SVM mäter avstånd mellan punkter.Om en kolumn har jättestora tal (t.ex. Lön: 30 000) och en annan har små tal (t.ex. Ålder: 25), 
# kommer SVM bara bry sig om lönen för att avstånden är större där.Därför krymper vi allt till 0–1 (Normalisering) så att Ålder och Lön blir lika viktiga. 
# Du bör se skillnad i resultatet här!3. Looparna (3 stycken i varandra)Vi testar alla kombinationer automatiskt:Original vs Normaliserad (2 varianter)Split 75/25 vs 60/40 (2 varianter)Linear vs RBF (2 varianter)
#Totalt blir det $2 \times 2 \times 2 = 8$ körningar.4. Koden inne i loopensvm_model = SVC(kernel=kernel_type): 
#Här skapar vi roboten och säger åt den: "Använd det raka snöret (linear)" eller "Använd det böjda snöret (rbf)".fit och predict: Precis som förra gången. 
#Roboten pluggar och sen gör den provet.5. ResultatetKoden spottar ut en lista på alla 8 försök och sen en topplista.Titta noga på topplistan: 
#Är det "Normaliserad" som vinner? (Oftast ja för SVM).Är det "RBF" eller "Linear" som vinner? (Beror på hur krånglig din data är).
#Tips till videon:När du spelar in, peka på topplistan och säg: "Här ser vi att SVM fungerade bäst när vi använde [Normaliserad data] och [RBF-kärnan]. Det beror nog på att..."